### Task 0 - Preparation of Task 1

A. Fetching the data

In [1]:
import requests
URL = 'https://collector-otp-prod.camsys-apps.com/realtime/gtfsrt/ALL/alerts?type=json'
data = requests.get(URL).json()

In [3]:
print('Number of alerts:', len(data['entity']))
print('Details of alrert indexed 75')
alert = data['entity'][75]['alert']
print('**active_period**')
display(alert['active_period'])
print('**informed_entity**')
display(alert['informed_entity'])
print('**alert_type**')
display(alert['transit_realtime.mercury_alert']['alert_type'])

Number of alerts: 398
Details of alrert indexed 75
**active_period**


[{'start': 1713326400, 'end': 1745193600}]

**informed_entity**


[{'agency_id': 'MTABC',
  'route_id': 'BXM3',
  'transit_realtime.mercury_entity_selector': {'sort_order': 'MTA:BXM3:17'}}]

**alert_type**


'Planned - Detour'

Use 'A' as a list of entities for step B

In [4]:
A = data['entity']

B. Traverse through the alerts, and keep those that is with agency_id of value 'MTASBWY' and has a route_id. Keep the 'alert' field in the output.

In [5]:
B = [entity['alert']
     for entity in A
     for x in entity['alert']['informed_entity']
     if 'route_id' in x and x['agency_id'] == 'MTASBWY']
print(len(B))
print('===')
display(B[0])
print('===')
display(B[1])

221
===


{'active_period': [{'start': 1729650816, 'end': 1729651530}],
 'informed_entity': [{'agency_id': 'MTASBWY',
   'route_id': 'N',
   'transit_realtime.mercury_entity_selector': {'sort_order': 'MTASBWY:N:26'}},
  {'agency_id': 'MTASBWY', 'stop_id': 'R09'}],
 'header_text': {'translation': [{'text': 'Astoria-bound [N] trains are running with delays after we requested NYPD assistance for someone being disruptive on a train at Queensboro Plaza.',
    'language': 'en'},
   {'text': '<p><strong>Astoria</strong>-bound [N] trains are running with delays after we requested NYPD assistance for someone being disruptive on a train at <b>Queensboro Plaza</b>.</p>',
    'language': 'en-html'}]},
 'transit_realtime.mercury_alert': {'created_at': 1729648962,
  'updated_at': 1729650816,
  'alert_type': 'Delays',
  'display_before_active': 0}}

===


{'active_period': [{'start': 1729904400, 'end': 1730102400}],
 'informed_entity': [{'agency_id': 'MTASBWY',
   'route_id': 'SI',
   'transit_realtime.mercury_entity_selector': {'sort_order': 'MTASBWY:SI:10'}},
  {'agency_id': 'MTASBWY', 'stop_id': 'S11'},
  {'agency_id': 'MTASBWY', 'stop_id': 'S15'}],
 'header_text': {'translation': [{'text': "All [SIR] trains from Arthur Kill to Prince's Bay board from the Tottenville-bound platform",
    'language': 'en'},
   {'text': "<p>All [SIR] trains from <b>Arthur Kill</b> to <b>Prince's Bay</b> board from the Tottenville-bound platform</p>",
    'language': 'en-html'}]},
 'description_text': {'translation': [{'text': "Boarding change includes Arthur Kill, Richmond Valley, Pleasant Plains and Prince's Bay stations.\nWhat's happening?\nScheduled maintenance",
    'language': 'en'},
   {'text': "<ul><li><p>Boarding change includes <b>Arthur Kill</b>, <b>Richmond Valley</b>, <b>Pleasant Plains</b> and <b>Prince's Bay</b> stations.</p></li></ul><p>

C. Filter the data to only include the active period based on the current time.

In [6]:
import pytz
from datetime import datetime
ET = pytz.timezone('US/Eastern')
now = datetime.now().astimezone(ET)

def is_active(alert):
    for p in alert['active_period']:
        start = datetime.fromtimestamp(p['start'], ET)
        end = datetime.fromtimestamp(p.get('end', now.timestamp()), ET)
        if start <= now <= end:
            return True
        
C = list(filter(is_active, B))
print(len(C))
print('===')
C[:2]

16
===


[{'active_period': [{'start': 1728354600, 'end': 1728378900},
   {'start': 1728959400, 'end': 1728983700},
   {'start': 1729564200, 'end': 1729588500},
   {'start': 1730169000, 'end': 1730193300},
   {'start': 1730777400, 'end': 1730801700},
   {'start': 1731382200, 'end': 1731406500},
   {'start': 1731987000, 'end': 1732011300},
   {'start': 1732591800, 'end': 1732616100},
   {'start': 1733196600, 'end': 1733220900},
   {'start': 1733801400, 'end': 1733825700},
   {'start': 1734406200, 'end': 1734430500},
   {'start': 1735011000, 'end': 1735035300},
   {'start': 1735615800, 'end': 1735640100},
   {'start': 1736220600, 'end': 1736244900},
   {'start': 1736825400, 'end': 1736849700},
   {'start': 1737430200, 'end': 1737454500},
   {'start': 1738035000, 'end': 1738059300},
   {'start': 1738639800, 'end': 1738664100},
   {'start': 1739244600, 'end': 1739268900},
   {'start': 1739849400, 'end': 1739873700},
   {'start': 1740454200, 'end': 1740478500},
   {'start': 1741059000, 'end': 174108

D. Only keep the alert_type and route_id fields as a list of tuples.

In [7]:
D = list(map(lambda x: (x['transit_realtime.mercury_alert']['alert_type'],
                        x['informed_entity'][0]['route_id']), C))
D

[('No Scheduled Service', 'B'),
 ('No Scheduled Service', 'W'),
 ('Station Notice', '1'),
 ('Planned - Stops Skipped', '1'),
 ('Planned - Express to Local', 'N'),
 ('Extra Service', '7'),
 ('Planned - Reroute', 'Q'),
 ('Planned - Reroute', 'R'),
 ('Planned - Part Suspended', 'N'),
 ('Planned - Stops Skipped', '1'),
 ('Planned - Part Suspended', '1'),
 ('No Scheduled Service', 'Z'),
 ('Station Notice', 'J'),
 ('Station Notice', 'J'),
 ('Station Notice', 'Q'),
 ('Station Notice', '7')]

E. Turn the above list into a dictionary where keys are 'alert_type' and values are the sets of 'route_id'. Also, insert a key of 'Non Active Alert' that contains the rest of the key to the list.

In [8]:
from collections import defaultdict
E = defaultdict(set)
no_active = set(list('1234567ABCDEFGJLMNQRSWZ')+['SI', 'SF', 'SR'])
for alert_type, route_id in D:
    E[alert_type].add(route_id)
    no_active.discard(route_id)
E['Non Active Alerts'] = no_active
E

defaultdict(set,
            {'No Scheduled Service': {'B', 'W', 'Z'},
             'Station Notice': {'1', '7', 'J', 'Q'},
             'Planned - Stops Skipped': {'1'},
             'Planned - Express to Local': {'N'},
             'Extra Service': {'7'},
             'Planned - Reroute': {'Q', 'R'},
             'Planned - Part Suspended': {'1', 'N'},
             'Non Active Alerts': {'2',
              '3',
              '4',
              '5',
              '6',
              'A',
              'C',
              'D',
              'E',
              'F',
              'G',
              'L',
              'M',
              'S',
              'SF',
              'SI',
              'SR'}})

F. Traverse the dictionary to output the status.

In [9]:
for status, lines in sorted(E.items()):
    print(status, ':', ' '.join(sorted(lines)))

Extra Service : 7
No Scheduled Service : B W Z
Non Active Alerts : 2 3 4 5 6 A C D E F G L M S SF SI SR
Planned - Express to Local : N
Planned - Part Suspended : 1 N
Planned - Reroute : Q R
Planned - Stops Skipped : 1
Station Notice : 1 7 J Q


In [28]:
class MTAFeed:
    def __init__(self):
        self.refresh_time = datetime.now()
        self.data = E  # Assuming E is the dictionary from previous cells

    def getRefreshTime(self):
        return self.refresh_time

    def items(self):
        return self.data.items()
    
    def __getitem__(self, key):
        return self.data[key]

feed = MTAFeed()
print('Last refresh:', feed.getRefreshTime().isoformat(' ')[:19])
for status, lines in feed.items():
    print(status, ':', ' '.join(sorted(lines)))
no_active = 'Non Active Alerts'
print(no_active, ':', ' '.join(sorted(feed[no_active])))

Last refresh: 2024-10-22 23:29:22
No Scheduled Service : B W Z
Station Notice : 1 7 J Q
Planned - Stops Skipped : 1
Planned - Express to Local : N
Extra Service : 7
Planned - Reroute : Q R
Planned - Part Suspended : 1 N
Non Active Alerts : 2 3 4 5 6 A C D E F G L M S SF SI SR
Non Active Alerts : 2 3 4 5 6 A C D E F G L M S SF SI SR


In [15]:
!python task1.py

### Task 1

Complete the implementation for the class below, and convert it to a standalone script to test.

In [66]:
from collections import defaultdict
from datetime import datetime
import pytz
import requests

class MTAFeed:
    '''
    This class fetches data from the MTA feed and parse them into active alerts
    based on the current time (or a time that it gives). The class should also
    act as a 'dict' allowing user to iterate and retrieve items, with a special
    key 'Non Active Alerts' to list lines that have no active alerts.
    Sample usage:

    >>> feed = MTAFeed()

    >>> print(feed.getLines())
    {'1', '2', '3', '4', '5', '6', '7', 'A', 'B', 'C',..., 'SI', 'SF', 'SR'}

    >>> for item in feed.items():
    ...     print(item)
    ('Delays', {'N', 'W'})
    ('Planned - Express to Local', {'Z', 'J'})
    ('Planned - Trains Rerouted', {'D'})
    ('Station Notice', {'Z', 'J'})

    >>> feed.refresh()
    >>> for item in feed.items(include_non_active=True):
    ...     print(item)
    ('Delays', {'N', 'W'})
    ('Non Active Alerts', {'1', 'C', '2', '5', 'G', ..., 'Q'})
    ('Planned - Express to Local', {'Z', 'J'})
    ('Planned - Trains Rerouted', {'D'})
    ('Station Notice', {'Z', 'J'})

    >>> print(feed['Non Active Alerts'])
    {'1', 'C', '2', '5', 'G', ..., 'Q'}

    '''
    def __init__(self):
        '''
        Initialize the class attributes and start retrieving the feed (by
        calling the class method self.refresh())
        '''
        self.refresh()

    def refresh(self):
        '''
        This method refreshes the data by downloading the feed and extracting
        the alert based on the current timestamp (needed for checking active periods).
        '''
        self.refresh_time = datetime.now(pytz.timezone('US/Eastern'))
        self.data = defaultdict(set)

        no_active = set(list('1234567ABCDEFGJLMNQRSWZ') + ['SI', 'SF', 'SR'])
        for alert_type, route_id in D:
            self.data[alert_type].add(route_id)
            no_active.discard(route_id)
        self.data['Non Active Alerts'] = no_active

    def getRefreshTime(self):
        '''
        This returns the datetime object of when we last refresh the feed. The
        datetime object should be timezone aware and in the same time zone of
        the feed, 'US/Eastern'

        >>> feed.getRefreshTime().isoformat()
        '2023-03-29T09:00:28.789415-04:00'

        '''
        # return self.refresh_time

        # Ensure the refresh time is timezone aware and in 'US/Eastern'
        if self.refresh_time.tzinfo is None:
            self.refresh_time = self.refresh_time.replace(tzinfo=pytz.timezone('US/Eastern'))
        return self.refresh_time
    
    def items(self, include_non_active=False):
        '''
        Returns an iterator to all alerts (including 'Non Active Alerts' if
        include_non_active is set to True). This should be a generator. The
        idea is to allow users to iterate through the alerts, e.g.:

        >>> for item in feed.items():
        ...

        '''
        for alert_type, routes in self.data.items():
            if include_non_active or alert_type != 'Non Active Alerts':
                yield alert_type, routes


    def __getitem__(self, alert_type):
        '''
        We override this built-in operator to allow users to directly retrieve
        the set of lines associated with the alert_type, e.g.:

        >>> print(feed['Delays'])
        {'N', 'W'}

        '''
        return self.data.get(alert_type, set())
    
    def getLines(self):
        '''
        Return the set of all possible lines, which should be the union of
        all lines across both active and non-active alerts.
        '''
        all_lines = set()

        # Iterate through all the alerts and collect all the lines
        for routes in self.data.values():
            all_lines.update(routes)

        return all_lines


Use the following code to test your class.

In [67]:
feed = MTAFeed()
print('Last refresh: ', feed.getRefreshTime().isoformat(' ')[:19])
for status, lines in feed.items():
    print(status, ':', ' '.join(sorted(lines)))
no_active = 'Non Active Alerts'
print(no_active, ':', ' '.join(sorted(feed[no_active])))

Last refresh:  2024-10-23 01:01:13
No Scheduled Service : B W Z
Station Notice : 1 7 J Q
Planned - Stops Skipped : 1
Planned - Express to Local : N
Extra Service : 7
Planned - Reroute : Q R
Planned - Part Suspended : 1 N
Non Active Alerts : 2 3 4 5 6 A C D E F G L M S SF SI SR


In [69]:
!python task1.py

Traceback (most recent call last):
  File "/Users/yemoe/Downloads/PyQt/task1.py", line 3, in <module>
    import pytz
ModuleNotFoundError: No module named 'pytz'


### Task 2

Integrate the MTAFeed in Step 1 into our existing GUI code. You may reuse task1.py code above as a module for your MTAFeed class:

In [ ]:
if __name__ == '__main__':
    feed = MTAFeed()
    print('Last refresh: ', feed.getRefreshTime().isoformat(' ')[:19])
    for status, lines in feed.items():
        print(status, ':', ' '.join(sorted(lines)))
no_active = 'Non Active Alerts'
print(no_active, ':', ' '.join(sorted(feed[no_active])))

Then, you can import MTAFeed in your GUI code without worrying the test being run.